### Contexte
Vous travaillez pour une entreprise de commerce électronique qui a collecté des données de ventes sur une année. Avant d'effectuer une analyse approfondie, l'entreprise souhaite s'assurer de la qualité et de la cohérence des données. Votre tâche est de créer un système de validation robuste en utilisant Pydantic v2.

### Objectifs
1. Créer des modèles Pydantic pour valider strictement les données de ventes.
2. Implémenter un système de validation qui identifie et catégorise les anomalies dans les données.
3. Générer un rapport détaillé sur la qualité des données.

### Données
Vous disposez d'un fichier CSV nommé "ventes_detaillees.csv", disponible en téléchargement sur [ce lien](http://sc-e.fr/wcs/ventes_detaillees.csv), contenant les colonnes suivantes :
id_vente, date, heure, id_produit, nom_produit, prix_produit, categorie_produit, quantite, pays, ville, methode_paiement, statut_commande, satisfaction_client

### Tâches

1. **Création des modèles Pydantic**
   - Créez un modèle `Produit` avec les contraintes suivantes :
     - `id`: doit commencer par 'P' suivi de 3 chiffres
     - `nom`: chaîne non vide
     - `prix`: nombre positif avec 2 décimales maximum
     - `categorie`: doit être l'une des catégories prédéfinies (Électronique, Vêtements, Alimentation, Maison, Loisirs, Beauté, Électroménager, Livres, Sport, Papeterie, Accessoires)

   - Créez un modèle `Vente` avec les contraintes suivantes :
     - `id`: doit commencer par 'V' suivi de 6 chiffres
     - `date` et `heure`: format valide
     - `produit`: utilise le modèle `Produit`
     - `quantite`: entier positif
     - `pays`: doit être l'un des pays européens prédéfinis (France, Allemagne, Espagne, Italie, Royaume-Uni, Belgique, Pays-Bas, Suisse, Portugal, Autriche)
     - `ville`: chaîne non vide
     - `methode_paiement`: doit être l'une des méthodes prédéfinies (Carte de crédit, PayPal, Virement bancaire, Apple Pay, Google Pay)
     - `statut_commande`: doit être l'un des statuts prédéfinis (En cours de traitement, Expédiée, Livrée, Annulée)
     - `satisfaction_client`: entier entre 1 et 5

2. **Système de validation**
   - Implémentez une fonction qui lit le CSV et valide chaque ligne.
   - Catégorisez les erreurs (par exemple : erreur de format, valeur hors limites, valeur manquante).
   - Comptabilisez les occurrences de chaque type d'erreur.

3. **Rapport de qualité des données**
   - Générez un rapport indiquant :
     - Le nombre total d'enregistrements traités
     - Le nombre d'enregistrements valides
     - Le nombre et le pourcentage d'enregistrements invalides
     - Un résumé des types d'erreurs rencontrées et leur fréquence
     - Des exemples d'enregistrements invalides pour chaque type d'erreur

### Code de départ

In [33]:
from pydantic import BaseModel, Field, field_validator, ValidationError
from typing import List
from datetime import date, time
import pandas as pd

def valider_donnees_ventes(chemin_fichier: str) -> tuple:
    # Implémentez la lecture du CSV et la validation ici
    pass

def generer_rapport_qualite(resultats_validation: tuple):
    # Implémentez la génération du rapport ici
    pass

if __name__ == "__main__":
    resultats = valider_donnees_ventes("ventes_detaillees")
    generer_rapport_qualite(resultats)

### Conseils
- Utilisez les fonctionnalités de Pydantic comme `Field`, `field_validator` pour définir des contraintes complexes.
- Pensez à gérer les cas particuliers comme les valeurs manquantes ou mal formatées.
- Pour le rapport, essayez de fournir des informations utiles qui aideraient à comprendre la nature et l'étendue des problèmes de qualité des données.
- N'oubliez pas de traiter les dates et heures correctement lors de la lecture du CSV.

Bonne chance !

In [34]:
from collections import defaultdict


errors = defaultdict(list)
errors["toot"] = ["hello", "world"]

errors["toot"].append("totot")
print(errors)

defaultdict(<class 'list'>, {'toot': ['hello', 'world', 'totot']})


In [35]:
"""
id: doit commencer par 'P' suivi de 3 chiffres
nom: chaîne non vide
prix: nombre positif avec 2 décimales maximum
categorie: doit être l'une des catégories prédéfinies (Électronique, Vêtements, Alimentation, Maison, Loisirs, Beauté, Électroménager, Livres, Sport, Papeterie, Accessoires)
"""

class Produit(BaseModel):
  id: str = Field(..., pattern=r"^P\d{3}")
  name: str = Field(..., min_length=1)
  price: float = Field(..., gt=0)
  category: str

  @field_validator('category')
  @classmethod
  def valid_category(cls, value: str) -> str:
    cat = ["Électronique", "Vêtements", 'Alimentation', 'Maison', "Loisirs", "Beauté", "Électroménager", "Livres", "Sport", 'Papeterie', 'Accessoires']
    if value not in cat:
      raise ValueError(f"The category should be: {''.join(cat)}")
    return value

  @field_validator('price')
  @classmethod
  def valid_price(cls, price: float) -> float:
    return round(price, 2)





In [36]:
'''
id: doit commencer par 'V' suivi de 6 chiffres
date et heure: format valide
produit: utilise le modèle Produit
quantite: entier positif
pays: doit être l'un des pays européens prédéfinis (France, Allemagne, Espagne, Italie, Royaume-Uni, Belgique, Pays-Bas, Suisse, Portugal, Autriche)
ville: chaîne non vide
methode_paiement: doit être l'une des méthodes prédéfinies (Carte de crédit, PayPal, Virement bancaire, Apple Pay, Google Pay)
statut_commande: doit être l'un des statuts prédéfinis (En cours de traitement, Expédiée, Livrée, Annulée)
satisfaction_client: entier entre 1 et 5
'''

# Vente

class Vente(BaseModel):
  id: str = Field(..., pattern=r'^V\d{6}')
  date: date
  time: time
  produit: Produit
  quantity: int = Field(..., gt=0)
  country: str
  city: str = Field(..., min_length=1)
  payment: str
  order_status: str
  satisfaction: int = Field(..., ge=1, le=5)

  @field_validator("country")
  @classmethod
  def valid_country(cls, value: str) -> str:
    valid_countries = ['France', 'Allemagne', 'Espagne', 'Italie', 'Royaume-Uni', 'Belgique', 'Pays-Bas', 'Suisse', 'Portugal','Autriche']
    if value not in valid_countries:
      raise ValueError(f"Country must be one of the list: {','.join(valid_countries)}")
    return value

  @field_validator("payment")
  @classmethod
  def valid_payment_method(cls, value: str) -> str:
    valid_methods = ['Carte de crédit', 'PayPal', 'Virement bancaire', 'Apple Pay', 'Google Pay']
    if value not in valid_methods:
      raise ValueError(f"Payment must be: {','.join(valid_methods)}")
    return value

  @field_validator('order_status')
  @classmethod
  def valid_status(cls, value: str) -> str:
    valid_status = ['En cours de traitement', 'Expédiée', 'Livrée', 'Annulée']
    if value not in valid_status:
      raise ValueError(f"The status must be: {','.join(valid_status)}")
    return value




In [37]:
'''
Implémentez une fonction qui lit le CSV et valide chaque ligne.
Catégorisez les erreurs (par exemple : erreur de format, valeur hors limites, valeur manquante).
Comptabilisez les occurrences de chaque type d'erreur.
'''

df = pd.read_csv("http://sc-e.fr/wcs/ventes_detaillees.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_vente             10000 non-null  object 
 1   date                 10000 non-null  object 
 2   heure                10000 non-null  object 
 3   id_produit           10000 non-null  object 
 4   nom_produit          10000 non-null  object 
 5   prix_produit         10000 non-null  float64
 6   categorie_produit    10000 non-null  object 
 7   quantite             10000 non-null  int64  
 8   pays                 10000 non-null  object 
 9   ville                10000 non-null  object 
 10  methode_paiement     10000 non-null  object 
 11  statut_commande      10000 non-null  object 
 12  satisfaction_client  10000 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1015.8+ KB


In [48]:
def read_and_validate(file):
  df = pd.read_csv(file)
  all_errors = defaultdict(list)
  total_items = 0
  invalid_items = []

  for index, vente in df.iterrows():
      total_items +=1
      try:
          produit = Produit(id = vente['id_produit'],
                            name=vente['nom_produit'],
                            price = vente['prix_produit'],
                            category = vente['categorie_produit']
                            )
          value = Vente(
          id = vente['id_vente'],
          date = vente['date'],
          time = vente['heure'],
          produit = produit,
          quantity = vente['quantite'],
          country = vente['pays'],
          city = vente['ville'],
          payment = vente['methode_paiement'],
          order_status = vente['statut_commande'],
          satisfaction = vente['satisfaction_client']
          )

      except ValidationError as e:
         invalid_items.append(index)
         for error in e.errors():
            all_errors[error['type']].append([error['loc'][0], index])
  occurences_for_type_error = {}
  for key, value in all_errors.items():
    occurences_for_type_error[key] = len(value)

  return total_items, len(invalid_items), occurences_for_type_error, all_errors





read_and_validate("http://sc-e.fr/wcs/ventes_detaillees.csv")

(10000,
 1000,
 {'greater_than': 329,
  'value_error': 339,
  'string_pattern_mismatch': 160,
  'less_than_equal': 172},
 defaultdict(list,
             {'greater_than': [['price', 1],
               ['quantity', 20],
               ['price', 27],
               ['price', 35],
               ['price', 37],
               ['quantity', 111],
               ['quantity', 156],
               ['quantity', 158],
               ['price', 167],
               ['price', 173],
               ['quantity', 201],
               ['price', 232],
               ['quantity', 256],
               ['price', 274],
               ['price', 359],
               ['price', 369],
               ['price', 382],
               ['quantity', 408],
               ['price', 436],
               ['quantity', 484],
               ['quantity', 490],
               ['price', 571],
               ['quantity', 572],
               ['quantity', 600],
               ['price', 656],
               ['price', 669],
           

In [53]:
'''
Le nombre total d'enregistrements traités
Le nombre d'enregistrements valides
Le nombre et le pourcentage d'enregistrements invalides
Un résumé des types d'erreurs rencontrées et leur fréquence
Des exemples d'enregistrements invalides pour chaque type d'erreur
'''

def quality_report(infos: tuple):
  total_items, invalid_items, freq_by_error, errors_index = infos

  print(f"Total number of items: {total_items} ")
  print(f"Number of valid items: {total_items - invalid_items}")
  print(f"Number of invalid items: {invalid_items}")
  percent_invalid = (invalid_items / total_items) * 100
  print(f"Percent of invalid items: {percent_invalid}")
  print(f"Resume frequency of errors")
  for key, value in freq_by_error.items():
    print(f"For the error {key}, there are {value} invalid rows")
    print(f"example of errors")
    some_index = errors_index[key][0:4]
    for i in some_index:
      print(f"Column: {i[0]}")
      print(dict(df.iloc[i[1]]))


result = read_and_validate("http://sc-e.fr/wcs/ventes_detaillees.csv")
quality_report(result)

Number of items: 10000
Number of valid items: 9000
Number of invalid items: 1000
Percent of invalid items: 10.0
Resume frequency of errors
For the error greater_than, there are 329 invalid rows
example of errors
Column: price
{'id_vente': 'V000002', 'date': '2023-05-24', 'heure': '02:06:00', 'id_produit': 'P001', 'nom_produit': 'Smartphone', 'prix_produit': -499.99, 'categorie_produit': 'Électronique', 'quantite': 5, 'pays': 'France', 'ville': 'Ville_18_France', 'methode_paiement': 'Apple Pay', 'statut_commande': 'En cours de traitement', 'satisfaction_client': 5}
Column: quantity
{'id_vente': 'V000021', 'date': '2023-02-14', 'heure': '12:59:00', 'id_produit': 'P003', 'nom_produit': 'Livre de cuisine', 'prix_produit': 24.99, 'categorie_produit': 'Livres', 'quantite': 0, 'pays': 'Belgique', 'ville': 'Ville_5_Belgique', 'methode_paiement': 'Apple Pay', 'statut_commande': 'Livrée', 'satisfaction_client': 5}
Column: price
{'id_vente': 'V000028', 'date': '2023-09-04', 'heure': '00:37:00', '